<a href="https://colab.research.google.com/github/IshtiSikder/two_stage_template/blob/main/Two_stage_Stochastic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#checked

import time
t1 = time.perf_counter()

In [ ]:
#checked

#TASK
#write justifications for the weights 
#on paper goes under modelling section
'''
for i in [1,5,10,15,20]:
  for j in [1,5,10,15,20]:
    for k in [0.5,1,1.5]:
'''
L1 = 50
mismatch = 10
const = 0.5

In [ ]:
#checked

#Install gurobi
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#checked

#Import necessary libraries
import sys
import numpy as np
import pandas as pd
from numpy import linalg as la
import gurobipy as gp
from gurobipy import *

In [ ]:
#checked

#Activate gurobi license (only if run on google colab)
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', 'dcee8c55-6742-4816-bae1-a0742c8f71a9')
e.setParam('WLSSECRET', '70f89e9f-2426-41b9-abe7-88c4b6090a45')
e.setParam('LICENSEID',820825)
e.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 820825
Academic license - for non-commercial use only - registered to isikder@uncc.edu


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=820825>

In [ ]:
#checked-

#Setup the scheduling model
TwoStage_Model = gp.Model("TwoStage",env=e) #env is set only if run on Google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/GOOGLE_COLAB/MAYO'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GOOGLE_COLAB/MAYO


In [ ]:
#checked

#PARAMETRES

#P(i,n): Number of patients of type i ∈ I in scenario n ∈ N
P = pd.read_excel("data.xlsx", sheet_name = "influx")
P = P.set_index('Date')
P = P.astype(int)
P.index = P.index.astype(int)
P.columns = P.columns.astype(int)
P = P.transpose()
P = P.drop(P.columns[60:],axis=1)


#S(t): Staff level at time t ∈ T
S = pd.read_excel("data.xlsx", sheet_name = "StaffLevel")
S = S.set_index('Day')
S = S.astype(int)
S.index = S.index.astype(int)


#U(i,t): Maximum number of appointments of type i that can be scheduled at time t ∈ T
U = pd.read_excel("data.xlsx", sheet_name = "MaximumAppts")
U = U.set_index('Time slot')
U = U.astype(int)
U.index = U.index.astype(int)
U.columns = U.columns.astype(int)
U = U.transpose()

In [ ]:
#checked

#P = P.drop([240,300,360])
#P = P.drop(P.columns[10:],axis=1)
P

Date,1,2,3,4,5,6,7,8,9,10,...,51,52,53,54,55,56,57,58,59,60
30,24,22,25,21,26,16,18,28,21,21,...,22,21,20,28,26,27,21,18,19,17
60,7,16,9,12,7,12,10,7,7,8,...,6,8,7,6,6,9,8,11,10,7
120,8,9,8,4,7,9,6,8,6,11,...,6,8,6,7,7,10,7,10,8,9
180,12,9,8,10,14,6,10,7,9,10,...,10,12,13,7,7,8,8,8,10,8
240,3,2,7,4,3,3,3,7,8,5,...,5,7,7,2,5,3,6,5,4,6
300,0,0,0,1,2,2,1,1,0,2,...,0,0,2,2,2,1,1,2,2,3
360,4,2,3,1,0,2,2,0,1,1,...,2,2,1,3,1,1,2,2,2,0


In [ ]:
#checked

#U = U.drop([240,300,360])
U

Time slot,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
30,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
60,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
120,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
180,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
240,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
300,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
360,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [ ]:
#checked

#TASKS
##40 time slots, check new file

S

,Staff Level
Day,
1,2
2,2
3,3
4,3
5,4
6,4
7,5
8,5
9,7


In [ ]:
#checked

#SETS

#Total length of template in minutes
H = 40*15
print('H:',H)

#Length of basic time slot
h = 15 
print('h:',h)

#Set of time slots
T = list(range(1,int(H/h)+1))
print('Total time slots (length of T):',len(T))

#Set of patient types
I = {30,60,120,180,240,300,360}
print('I:',I)

#Set of scenarios (i.e 22 day patient influx)
N = list(range(P.columns[0],P.columns[-1]+1))
print('Total no of scenarios (length of N):',len(N))

#Maximum number of beds available for any time block
cap = 14 
print('cap:',cap)

#Maximum number of beds available for time block 1
cap1 = 7
print('cap1:',cap1)

###New edits
#bigM
bigM = 3*max(S['Staff Level'])
print('bigM:',bigM)
############

H: 600
h: 15
Total time slots (length of T): 40
I: {360, 300, 240, 180, 120, 60, 30}
Total no of scenarios (length of N): 60
cap: 14
cap1: 7
bigM: 21


In [ ]:
lst = []

for i in I:
  for ii in I:
    if i - ii > 0:
      lst.append(i-ii)

lst = set(lst)
lst = sorted(lst)

L2 = dict()

count = 0

for key in lst:
  count += 1
  L2[key] = count

In [ ]:
lst = []

for i in I:
  for ii in I:
    for iii in I:
      if ii < i and iii < i and ii + iii >= i:
        lst.append(ii+iii-i)

lst = set(lst)
lst = sorted(lst)

L3 = dict()

count = 0

for key in lst:
  count += 1
  L3[key] = count

In [ ]:
lst = []

for i in I:
  for ii in I:
    for iii in I:
      if i>=ii + iii:
        lst.append(i-(ii+iii))

lst = set(lst)
lst = sorted(lst)

L4 = dict()

count = 0

for key in lst:
  count += 1
  L4[key] = count

In [ ]:
lam1 = TwoStage_Model.addVar(lb = 0,vtype = GRB.INTEGER)
lam2 = TwoStage_Model.addVar(lb = 0, vtype = GRB.INTEGER)

In [ ]:
#VARIABLES 

# x(i,t): Number of started appointments of type i ∈ I at time t ∈ T 
#(that can be finished before 6 pm)
aux_idx_x = {
            (i,t) for i in I for t in T if t + (i/h) <= (H/h) + 1 
        }
x = TwoStage_Model.addVars(aux_idx_x, lb=0,vtype=GRB.INTEGER, name='x(i,t)')


# y(i,t): Total number of appointments of type i ∈ I at time t ∈ T
y = TwoStage_Model.addVars(I,T, lb=0,vtype=GRB.INTEGER, name='y(i,t)')

# z(i,ii,j,t,n): Binary variable, 1 if the jth appointment of type i at time t is 
# used to treat a patient of type ii(ii<=i) in scenario n ; and 0 otherwise
aux_idx_z = {
            (i,ii,j,t,n) for i,t in aux_idx_x for ii in I for j in range(1,int(U.loc[i,t]+1)) for n in N if i>=ii 
        }
z = TwoStage_Model.addVars(aux_idx_z, lb=0,vtype=GRB.BINARY, name='z(i,ii,j,t,n)')

# w(i,ii,iii,j,t,n): Binary variable, 1 if the jth appointment of type i at time t is 
# broken into two slots to treat patients of type ii and type iii (ii + iii<=i) in scenario n ; and 0 otherwise
aux_idx_w = {
            (i,ii,iii,j,t,n) for i,t in aux_idx_x for ii in I for iii in I for j in range(1,int(U.loc[i,t]+1)) for n in N if i>=ii + iii
        }
w = TwoStage_Model.addVars(aux_idx_w, lb=0,vtype=GRB.BINARY, name='w(i,ii,iii,j,t,n)')


# u(i,t,n): Number of patients of type i that start the treatment at time t in scenario n
u = TwoStage_Model.addVars(I,T,N, lb=0, name='u(i,t,n)')

# v(i,j,t): Binary variable, 1 if the jth appointment of type i at time t is active; 0 otherwise.
idx_v = {
            (i,j,t) for i,t in aux_idx_x for j in range(1,int(U.loc[i,t]+1))
        }
v = TwoStage_Model.addVars(idx_v, lb=0,vtype=GRB.BINARY, name='v(i,j,t)')

#q(i,n)
q = TwoStage_Model.addVars(I,N,lb=0,name='q(i,n)')

#g(i,t,n): Total number of patients of type i at time t in scenario n
g = TwoStage_Model.addVars(I,T,N, lb=0, name='u(i,t,n)')



#####July 15th
'''
#b(t,n): staff level violation at time block t for scenario n
b = TwoStage_Model.addVars(T, N, vtype=GRB.INTEGER, name = 'b(t,n)')
'''
#######



######New edits
vio = TwoStage_Model.addVars(T, vtype=GRB.INTEGER, name = 'vio(t)')
############


######July 15th
f = TwoStage_Model.addVars(I, T, N, lb = 0, name = 'f(i,t)')
###############


#July 15th
'''
bb = TwoStage_Model.addVars(T, N, vtype=GRB.INTEGER, name = 'bb(t,n)')
'''
#########


######New edits
vio2 = TwoStage_Model.addVars(T, vtype=GRB.INTEGER, name = 'vio2(t)')
############




#combining two slots

aux_idx_o = {
            (i,ii,iii,t,j1,j2,n) for i,t in aux_idx_x for ii in I for iii in I if t+ii/h in T
            for j1 in range(1,int(U.loc[ii,t])+1) for j2 in range(1,int(U.loc[iii,t+ii/h])+1) for n in N
            if ii < i and iii < i and ii + iii >= i
        }

#Original override code has the vtype set to INTEGER. I changed it to BINARY as described in the override paper
o = TwoStage_Model.addVars(aux_idx_o, vtype=GRB.BINARY, name='o')

aux_idx_block1 = {
        (i,ii,t,j,n) for i in I for ii in I for t in T for j in range(1,int(U.loc[ii,t])+1) for n in N if i > ii
        }

block1 = TwoStage_Model.addVars(aux_idx_block1, vtype=GRB.BINARY, name='block1')

aux_idx_block2 = {
        (i,iii,t,j,n) for i in I for iii in I for t in T for j in range(1,int(U.loc[iii,t])+1) for n in N if i > iii
        }

block2 = TwoStage_Model.addVars(aux_idx_block2, vtype=GRB.BINARY, name='block2')


#July 25th edit

v_over = TwoStage_Model.addVars(I,N,lb = 0,name = 'v_over[i]')

v_under = TwoStage_Model.addVars(I,N,lb = 0,name = 'v_under[i]')

KeyboardInterrupt: ignored

In [ ]:
TwoStage_Model.addConstr(lam1 == const*vio.sum())
TwoStage_Model.addConstr(lam2 == const*vio2.sum())

In [ ]:
TwoStage_Model.addConstrs(v_over[i,n] - v_under[i,n] == x.sum(i,'*') - P.loc[i,n] for i in I for n in N)

In [ ]:
#override paper, constr 2 (updated with scenario n)

TwoStage_Model.addConstrs(
        o[i,ii,iii,t,j1,j2,n] <= block1[i,ii,t,j1,n] for (i,ii,iii,t,j1,j2,n) in aux_idx_o
        )

In [ ]:
#override paper, constr 3 (updated with scenario n)

TwoStage_Model.addConstrs(
        o[i,ii,iii,t,j1,j2,n] <= block2[i,iii,t + int(ii/h),j2,n] for (i,ii,iii,t,j1,j2,n) in aux_idx_o
        )


In [ ]:
#override paper, constr 4 (updated with scenario n)

TwoStage_Model.addConstrs(
        o[i,ii,iii,t,j1,j2,n] >= block1[i,ii,t,j1,n] + block2[i,iii,t + int(ii/h),j2,n] - 1 for (i,ii,iii,t,j1,j2,n) in aux_idx_o
        )

In [ ]:
#override paper, constr 6 (updated with scenario n)

TwoStage_Model.addConstrs(
        o.sum(i,ii,'*',t,j1,'*',n) <= 1 for (i,ii,t,j1,n) in aux_idx_block1)

In [ ]:
#override paper, constrs 5 (updated with n)

for i in I:
  for iii in I:
    for t in T:
      for j2 in range(1,int(U.loc[iii,t])+1):
        for n in N:
          if (i,iii,t,j2,n) in aux_idx_block2:
            expr = 0
            for ii in I:
              if t-int(ii/h) in T:
                for j1 in range(1,int(U.loc[ii,t-int(ii/h)])+1):
                  if (i,ii,iii,t-int(ii/h),j1,j2,n) in aux_idx_o:
                    expr = expr + o[i,ii,iii,t-int(ii/h),j1,j2,n]
            TwoStage_Model.addConstr(expr <= 1)

In [ ]:
# Constraint : x(i,t) <= U(i,t)

TwoStage_Model.addConstrs(x[i,t] <= U.loc[i,t] for i,t in aux_idx_x)

In [ ]:
#constr 1 (from paper)

TwoStage_Model.addConstrs(
    y[i,t] == quicksum(
        x[i, (t - tt)] for tt in list(range(int(i/h))) if t - tt> 0 if (i,t-tt) in aux_idx_x
    ) for i in I for t in T
)

In [ ]:
#constr 2 (from paper)

TwoStage_Model.addConstrs(y.sum('*',t) <= cap for t in T)

In [ ]:
#Constraint : sum of y <= cap1 for 1st time block

TwoStage_Model.addConstr(y.sum('*',1) <= cap1 )

In [ ]:
#constr 3 (from paper)

TwoStage_Model.addConstrs(x[i,t]== v.sum(i,'*',t) for i,t in aux_idx_x)

In [ ]:
#constr 4 (UPDATED)

for (i,j,t) in idx_v:
  for n in N:
    expr = 0
    for ii in I:
      if (i,ii,j,t,n) in aux_idx_z:
        expr = expr + z[i,ii,j,t,n]
    for ii in I:
      for iii in I:
        if (i,ii,iii,j,t,n) in aux_idx_w:
          expr = expr + w[i,ii,iii,j,t,n] 
    
    for ii in I:
      if (i,ii,t,j,n) in aux_idx_block1:
        expr = expr + block1[i,ii,t,j,n]

    for iii in I:
      if (i,iii,t,j,n) in aux_idx_block2:
        expr = expr + block2[i,iii,t,j,n]
    
    TwoStage_Model.addConstr(
        expr <= v[i,j,t]
        )

In [ ]:
#constr 5 updated

for (i,t,n) in u:
    expr = 0
    for ii in I:
      for j in range(1,int(U.loc[ii,t])+1):
        if (ii,i,j,t,n) in aux_idx_z:
          expr = expr + z[ii,i,j,t,n]
    
    for ii in I:
      for iii in I:
        for j in range(1,int(U.loc[ii,t])+1):
          if (ii,i,iii,j,t,n) in aux_idx_w:
            expr = expr + w[ii,i,iii,j,t,n] 
    for ii in I:
      for iii in I:
        for j in range(1,int(U.loc[ii,t])+1):
          if (ii,iii,i,j,t-(iii/h),n) in aux_idx_w:
            expr = expr + w[ii,iii,i,j,t-(iii/h),n]

    for ii in I:
      for iii in I:
        for j1 in range(1,int(U.loc[ii,t])+1):
          if t + (ii/h) in T:
            for j2 in range(1,int(U.loc[iii,t+(ii/h)])+1):
              if (i,ii,iii,t,j1,j2,n) in aux_idx_o:
                expr = expr + o[i,ii,iii,t,j1,j2,n]

    TwoStage_Model.addConstr(
        expr == u[i,t,n]
        )

In [ ]:
#constr 6 (from paper)

TwoStage_Model.addConstrs(
    g[i,t,n] == quicksum(
        u[i, (t - tt),n] for tt in list(range(int(i/h))) if t - tt > 0
    ) for i in I for t in T for n in N
)


In [ ]:
#constr 7 (from paper)

TwoStage_Model.addConstrs(q[i,n] >= P.loc[i,n] - u.sum(i,'*',n) for i in I for n in N)

In [ ]:
#contr 8 (from paper)

#####New edits
TwoStage_Model.addConstrs( 
    3*int(S.loc[t]) - y.sum('*', t) >= -bigM*(vio[t]) for t in T 
)
###############


In [ ]:
#July 15th
'''
TwoStage_Model.addConstrs(f[i, t, n] == u[i, t - i/h + 1, n] for i in I for t in T for n in N if t - i/h + 1 > 0)

TwoStage_Model.addConstrs(f[i, t, n] == 0 for i in I for t in T for n in N if t - i/h + 1 <= 0)
'''
##


###########New edits
e = TwoStage_Model.addVars(I,T,lb =0, vtype = GRB.INTEGER, name = 'e')

TwoStage_Model.addConstrs(e[i, t] == x[i, t - i/h + 1] for i in I for t in T if t - i/h + 1 > 0)

TwoStage_Model.addConstrs(e[i, t] == 0 for i in I for t in T if t - i/h + 1 <= 0)

TwoStage_Model.addConstrs(
    int(S.loc[t]) - (x.sum('*', t) + e.sum('*', t)) >= -bigM*(vio2[t]) for t in T for n in N
)
###############
 

In [ ]:

#July 15th
'''
TwoStage_Model.addConstrs( 
    b[t,n] == 3*int(S.loc[t]) - g.sum('*', t, n) for t in T for n in N
)

#1:1 persists
TwoStage_Model.addConstrs(
    bb[t,n] == int(S.loc[t]) - ( u.sum('*', t, n) + f.sum('*', t, n) ) for t in T for n in N
) 

'''
##


In [ ]:
#July 15th
'''
TwoStage_Model.addConstrs(
    u.sum(i, t, '*') / len(N) == x[i,t] for i,t in aux_idx_x 
) 

TwoStage_Model.addConstrs(
    g.sum(i, t, '*') / len(N) == y[i,t] for i,t in aux_idx_x 
)
'''
##

In [ ]:
TwoStage_Model.params.NonConvex = 2

In [ ]:
#Solving the objective function

######New edit
obj = (1/len(P.columns))*L1*(q.sum()) + lam2*vio2.sum() + lam1*vio.sum() + (1/len(P.columns))*mismatch*(v_under.sum()+ v_over.sum())
###################

#try to incorporate a term in the obj function that penalizes direct mismatch between template and influx data



#try to penalize violations in proportion to violation level

#July 15th
'''
obj = L1*q.sum() - lam2*bb.sum() - lam1*b.sum()
'''
##


for i,t in aux_idx_x:
  for ii in I:
    for j in range(1,int(U.loc[i,t]+1)):
      for n in N:
        if i>ii:
          obj += L2[i-ii]*z[i,ii,j,t,n]


for i,t in aux_idx_x:
  for ii in I:
    for iii in I:
      if t+ii/h in T:
        for j1 in range(1,int(U.loc[ii,t])+1):
          for j2 in range(1,int(U.loc[iii,t+ii/h])+1):
            for n in N:
              if ii < i and iii < i and ii + iii >= i:
                obj += L3[ii+iii-i]*o[i,ii,iii,t,j1,j2,n]

for i,t in aux_idx_x:
  for ii in I:
    for iii in I:
      for j in range(1,int(U.loc[i,t]+1)):
        for n in N:
          if i>=ii + iii:
            obj += L4[i-(ii+iii)]*w[i,ii,iii,j,t,n]


#obj = (1/len(P.columns))*obj


TwoStage_Model.setObjective(obj, GRB.MINIMIZE)
TwoStage_Model.optimize()

print('Obj: %g' % TwoStage_Model.ObjVal)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license - for non-commercial use only - registered to isikder@ncsu.edu
Optimize a model with 4076185 rows, 2395008 columns and 14032683 nonzeros
Model fingerprint: 0xe2371449
Model has 80 quadratic objective terms
Variable types: 51660 continuous, 2343348 integer (2342505 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+01]
  Objective range  [2e-01, 1e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 1.400000e+12
Presolve removed 273660 rows and 290460 columns (presolve time = 5s) ...
Presolve removed 279465 rows and 300219 columns (presolve time = 10s) ...
Presolve removed 279465 rows and 302769 columns (presolve time = 16s) ...
Presolve removed 282216 rows and 3

In [ ]:
L4

In [ ]:
#Printing out the template (x[i,t])

results = pd.DataFrame(index= T, columns = sorted(I))

for i,t in x.keys():
  results.loc[t,i] = int(x[i,t].X)

results = results.fillna(0)

results

In [ ]:
results.to_excel('/content/drive/MyDrive/GOOGLE_COLAB/MAYO/TWO_STAGE/two_stage_temp.xlsx')

In [ ]:
t2 = time.perf_counter()
print('time taken to run:',(t2-t1)/60,'minutes')

In [ ]:
'''
#Deactivating gurobi license (only for Google colab)

TwoStage_Model.dispose()
e.dispose()
'''

In [ ]:
#override paper, constr 2 print

class ovrConstr2():
  
  
  def constrs():
    count = 0
    for (i,ii,iii,t,j1,j2,n) in aux_idx_o:
      count += 1
      print('{}. o({},{},{},{},{},{},{}) <= block1({},{},{},{},{})'.format(count,i,ii,iii,t,j1,j2,n,i,ii,t,j1,n))
  
  
  def constr(type,block1,block2,time,apt1,apt2,scene):
    for (i,ii,iii,t,j1,j2,n) in aux_idx_o:
      if (i,ii,iii,t,j1,j2,n) == (type,block1,block2,time,apt1,apt2,scene):
        print('o({},{},{},{},{},{},{}) <= block1({},{},{},{},{})'.format(i,ii,iii,t,j1,j2,n,i,ii,t,j1,n))


#override paper, constr 3 print

class ovrConstr3():
  
  def constrs():
    count = 0
    for (i,ii,iii,t,j1,j2,n) in aux_idx_o:
      count += 1
      print('{}. o({},{},{},{},{},{},{}) <= block2({},{},{},{},{})'.format(count,i,ii,iii,t,j1,j2,n,i,iii,t + int(ii/h),j2,n))

  def constr(type,block1,block2,time,apt1,apt2,scene):
    for (i,ii,iii,t,j1,j2,n) in aux_idx_o:
      if (i,ii,iii,t,j1,j2,n) == (type,block1,block2,time,apt1,apt2,scene):
        print('o({},{},{},{},{},{},{}) <= block2({},{},{},{},{})'.format(i,ii,iii,t,j1,j2,n,i,iii,t + int(ii/h),j2,n))

#override paper, constr 4 print

class ovrConstr4():

  def constrs():
    count = 0
    for (i,ii,iii,t,j1,j2,n) in aux_idx_o:
      count += 1
      print('{}. o({},{},{},{},{},{},{}) >= block1({},{},{},{},{}) + block2({},{},{},{},{}) - 1'.format(count,i,ii,iii,t,j1,j2,n,i,ii,t,j1,n,i,iii,t + int(ii/h),j2,n))

  def constr(type,block1,block2,time,apt1,apt2,scene):
    for (i,ii,iii,t,j1,j2,n) in aux_idx_o:
      if (i,ii,iii,t,j1,j2,n) == (type,block1,block2,time,apt1,apt2,scene):
        print('o({},{},{},{},{},{},{}) >= block1({},{},{},{},{}) + block2({},{},{},{},{}) - 1'.format(i,ii,iii,t,j1,j2,n,i,ii,t,j1,n,i,iii,t + int(ii/h),j2,n))
  
#override paper, constr 6 print

class ovrConstr6():
  def constrs():
    count = 0
    lst = []
    for (i,ii,t,j1,n) in aux_idx_block1:
      count += 1
      for iii in I:
        for j2 in range(1,int(U.loc[iii,t])+1):
          lst.append('o({},{},{},{},{},{},{})'.format(i,ii,iii,t,j1,j2,n))
      print('{}. sum of({}) <= 1'.format(count,lst))

  def constr(type,block1,time,apt1,scene):
    lst = []
    for (i,ii,t,j1,n) in aux_idx_block1:
      if (i,ii,t,j1,n) == (type,block1,time,apt1,scene):
        for iii in I:
          for j2 in range(1,int(U.loc[iii,t])+1):
            lst.append('o({},{},{},{},{},{},{})'.format(i,ii,iii,t,j1,j2,n))
        print('sum of({}) <= 1'.format(lst))

#override paper, constr 5 print

class ovrConstr5():
  def constrs():
    count = 0
    for i in I:
      for iii in I:
        for t in T:
          for j2 in range(1,int(U.loc[iii,t])+1):
            for n in N:
              if (i,iii,t,j2,n) in aux_idx_block2:
                lst = []
                for ii in I:
                  if t-int(ii/h) in T:
                    for j1 in range(1,int(U.loc[ii,t-int(ii/h)])+1):
                      if (i,ii,iii,t-int(ii/h),j1,j2,n) in aux_idx_o:
                        lst.append('o({},{},{},{},{},{},{})'.format(i,ii,iii,t-int(ii/h),j1,j2,n))
                        count += 1
                print('{}. sum of({}) <= 1'.format(count,lst))

  def constr(type,block2,time,apt2,scene):
    for i in I:
      for iii in I:
        for t in T:
          for j2 in range(1,int(U.loc[iii,t])+1):
            for n in N:
              if (i,iii,t,j2,n) in aux_idx_block2:
                if (i,iii,t,j2,n) == (type,block2,time,apt2,scene):
                  lst = []
                  for ii in I:
                    if t-int(ii/h) in T:
                      for j1 in range(1,int(U.loc[ii,t-int(ii/h)])+1):
                        if (i,ii,iii,t-int(ii/h),j1,j2,n) in aux_idx_o:
                          lst.append('o({},{},{},{},{},{},{})'.format(i,ii,iii,t-int(ii/h),j1,j2,n))
                  print('sum of({}) <= 1'.format(lst))

# Constraint : x(i,t) <= U(i,t) print
 
class constrAdd1():
  def constrs():
    count = 0
    for i in I:
      for t in T:
        count += 1
        print("{}. x({},{}) <= {}".format(count,i,t,U.loc[i,t]))

  def constr(type,time):
    for i in I:
      if i == type:
        for t in T:
          if t == time:
            print("x({},{}) <= {}".format(i,t,U.loc[i,t]))

class constr1():
  def constrs():
    count = 0
    for i in I:
      for t in T:
        lst = []
        for tt in list(range(int(i/h))):
          if t - tt> 0:
            if (i,t-tt) in aux_idx_x:
              lst.append('x({},{})'.format(i,t-tt))
        count += 1
        print('{}. y({},{}) = sum of({})'.format(count,i,t,lst))

  def constr(type,time):
    for i in I:
      if i == type:
        for t in T:
          if t == time:
            lst = []
            for tt in list(range(int(i/h))):
              if t - tt> 0:
                if (i,t-tt) in aux_idx_x:
                  lst.append('x({},{})'.format(i,t-tt))
            print('y({},{}) = sum of({})'.format(i,t,lst))

#constr2

class constr2():
  def constrs():
    count = 0
    for t in T:
      lst = []
      for i in I:
        lst.append('y({},{})'.format(i,t))
      count += 1
      print('{}. sum of({}) <= {}'.format(count,lst,cap))

  def constr(time):
    for t in T:
      if t == time: 
        lst = []
        for i in I:
          lst.append('y({},{})'.format(i,t))
        print('sum of({}) <= {}'.format(lst,cap))

#Constraint : sum of y <= cap1 for 1st time block

class constrAdd2:
  def constrs():
    count = 0
    lst = []
    for t in T:
      if t == 1:
        for i in I:
          lst.append('y({},{})'.format(i,t))
      else:
        continue
    count += 1
    print('{}. sum of({}) <= {}'.format(count,lst,cap1))

class constr3():
  def constrs():
    count = 0
    for i in I:
      for t in T:
        lst = []
        for j in range(1,U.loc[i,t]+1):
          lst.append('v({},{},{})'.format(i,j,t))
        count += 1
        print('{}. x({},{}) = sum of({})'.format(count,i,t,lst))

  def constr(type,time):
    for i in I:
      if i == type:
        for t in T:
          if t == time:
            lst = []
            for j in range(1,int(U.loc[i,t])+1):
              lst.append('v({},{},{})'.format(i,j,t))
            print('x({},{}) = sum of({})'.format(i,t,lst))

#contr4 print updated

class constr4():
  def constrs():
    count = 0
    for (i,j,t) in idx_v:
      for n in N:
        lst1 = []
        lst2 = []
        lst3 = []
        lst4 = []
        for ii in I:
          if (i,ii,j,t,n) in aux_idx_z:
            lst1.append("z({},{},{},{},{})".format(i,ii,j,t,n))
        
        for ii in I:
          for iii in I:
            if (i,ii,iii,j,t,n) in aux_idx_w:
              lst2.append("w({},{},{},{},{},{})".format(i,ii,iii,j,t,n))

        for ii in I:
          if (i,ii,t,j,n) in aux_idx_block1:
            lst3.append('block1({},{},{},{},{})'.format(i,ii,t,j,n))

        for iii in I:
          if (i,iii,t,j,n) in aux_idx_block2:
            lst4.append('block2({},{},{},{},{})'.format(i,iii,t,j,n))

        count += 1
        print("{}. sum of({}) + sum of({}) + sum of({}) + sum of({}) <= v({},{},{})".format(count,lst1,lst2,lst3,lst4,i,j,t)) 



  def constr(type,apt,time,scene):
    for (i,j,t) in idx_v:
      for n in N:
        if (i,j,t,n) == (type,apt,time,scene):
          lst1 = []
          lst2 = []
          lst3 = []
          lst4 = []
          for ii in I:
            if (i,ii,j,t,n) in aux_idx_z:
              lst1.append("z({},{},{},{},{})".format(i,ii,j,t,n))
        
          for ii in I:
            for iii in I:
              if (i,ii,iii,j,t,n) in aux_idx_w:
                lst2.append("w({},{},{},{},{},{})".format(i,ii,iii,j,t,n))

          for ii in I:
            if (i,ii,t,j,n) in aux_idx_block1:
              lst3.append('block1({},{},{},{},{})'.format(i,ii,t,j,n))

          for iii in I:
            if (i,iii,t,j,n) in aux_idx_block2:
              lst4.append('block2({},{},{},{},{})'.format(i,iii,t,j,n))

          print("sum of({}) + sum of({}) + sum of({}) + sum of({}) <= v({},{},{})".format(lst1,lst2,lst3,lst4,i,j,t)) 

#constr 5 print updated


class constr5():
  def constrs():
    count = 0
    for (i,t,n) in u:
      lst1 = []
      lst2 = []
      lst3 = []
      lst4 = []

      for ii in I:
        for j in range(1,int(U.loc[ii,t])+1):
          if (ii,i,j,t,n) in aux_idx_z:
            lst1.append('z({},{},{},{},{})'.format(ii,i,j,t,n))
      
      for ii in I:
        for iii in I:
          for j in range(1,int(U.loc[ii,t])+1):
            if (ii,i,iii,j,t,n) in aux_idx_w:
              lst2.append('w({},{},{},{},{},{})'.format(ii,i,iii,j,t,n))
      
      for ii in I:
        for iii in I:
          for j in range(1,int(U.loc[ii,t])+1):
            if (ii,iii,i,j,t-(iii/h),n) in aux_idx_w:
              lst3.append('w({},{},{},{},{},{})'.format(ii,iii,i,j,t-(iii/h),n))

      for ii in I:
        for iii in I:
          for j1 in range(1,int(U.loc[ii,t])+1):
            if t + (ii/h) in T:
              for j2 in range(1,int(U.loc[iii,t+(ii/h)])+1):
                if (i,ii,iii,t,j1,j2,n) in aux_idx_o:
                  lst4.append(o[i,ii,iii,t,j1,j2,n])


      count += 1
      print('{}. sum of ({}) + sum of ({}) + sum of ({}) + sum of ({}) = u({},{},{})'.format(count,lst1,lst2,lst3,lst4,i,t,n))


  def constr(type,time,scene):
    for (i,t,n) in u:
      if (i,t,n) == (type,time,scene):
        lst1 = []
        lst2 = []
        lst3 = []
        lst4 = []

        for ii in I:
          for j in range(1,int(U.loc[ii,t])+1):
            if (ii,i,j,t,n) in aux_idx_z:
              lst1.append('z({},{},{},{},{})'.format(ii,i,j,t,n))
      
        for ii in I:
          for iii in I:
            for j in range(1,int(U.loc[ii,t])+1):
              if (ii,i,iii,j,t,n) in aux_idx_w:
                lst2.append('w({},{},{},{},{},{})'.format(ii,i,iii,j,t,n))
      
        for ii in I:
          for iii in I:
            for j in range(1,int(U.loc[ii,t])+1):
              if (ii,iii,i,j,t-(iii/h),n) in aux_idx_w:
                lst3.append('w({},{},{},{},{},{})'.format(ii,iii,i,j,t-(iii/h),n))

        for ii in I:
          for iii in I:
            for j1 in range(1,int(U.loc[ii,t])+1):
              if t + (ii/h) in T:
                for j2 in range(1,int(U.loc[iii,t+(ii/h)])+1):
                  if (i,ii,iii,t,j1,j2,n) in aux_idx_o:
                    lst4.append('o({},{},{},{},{},{},{})'.format(i,ii,iii,t,j1,j2,n))

        print('sum of ({}) + sum of ({}) + sum of ({}) + sum of ({}) = u({},{},{})'.format(lst1,lst2,lst3,lst4,i,t,n))

class constr6():
  def constrs():
    count = 0
    for i in I:
      for t in T:
        for n in N:
          lst = []
          for tt in list(range(int(i/h))):
            if t - tt > 0:
              lst.append('u({},{},{})'.format(i,t-tt,n))
          count += 1
          print('{}. g({},{},{}) = sum of({})'.format(count,i,t,n,lst))

  def constr(type,time,scene):
    for i in I:
      for t in T:
        for n in N:
          if (i,t,n) == (type,time,scene):
            lst = []
            for tt in list(range(int(i/h))):
              if t - tt > 0:
                lst.append('u({},{},{})'.format(i,t-tt,n))
            print('g({},{},{}) = sum of({})'.format(i,t,n,lst))   

class constr7():
  def constrs():
    count = 0
    for i in I:
      for n in N:
        lst = []
        for t in T:
          lst.append('u({},{},{})'.format(i,t,n))
        count += 1
        print('{}. q({},{}) >= P({},{}) - sum of({})'.format(count,i,n,i,n,lst))

  def constr(type,scene):
    for i in I:
      for n in N:
        if (i,n) == (type,scene):
          lst = []
          for t in T:
            lst.append('u({},{},{})'.format(i,t,n))
          print('q({},{}) >= P({},{}) - sum of({})'.format(i,n,i,n,lst))

class constrFinish():

  def constrs():
    count = 0
    for i in I:
      for t in T:
        for n in N:
          if t - i/h + 1 > 0:
            count += 1
            print('{}. f({},{},{}) = u({},{},{})'.format(count,i,t,n,i,(t-i/h+1),n))
          else:
            count += 1
            print('{}. f({},{},{}) = 0'.format(count,i,t,n))


  def constr(type,time,scene):
    for i in I:
      for t in T:
        for n in N:
          if (i,t,n) == (type,time,scene):
            if t - i/h + 1 > 0:
              print('f({},{},{}) == u({},{},{})'.format(i,t,n,i,(t-i/h+1),n))
            else:
              print('f({},{},{}) = 0'.format(i,t,n))

class constrVln1():
  def constrs():
    count = 0
    lst_u = []
    lst_f = []
    for t in T:
      for n in N:
        for i in I:
          lst_u.append('u({},{},{})'.format(i,t,n))
          lst_f.append('f({},{},{})'.format(i,t,n))
        print('{}. {} - sum of({}) + sum of ({}) >= -({})*(bb({},{}))'.format(count,int(S.loc[t]),lst_u,lst_f,bigM,t,n))

  def constr(time,scene):
    count = 0
    lst_u = []
    lst_f = []
    for t in T:
      for n in N:
        if (t,n) == (time,scene):
          for i in I:
            lst_u.append('u({},{},{})'.format(i,t,n))
            lst_f.append('f({},{},{})'.format(i,t,n))
          print('{} - sum of({}) + sum of ({}) >= -({})*(bb({},{}))'.format(int(S.loc[t]),lst_u,lst_f,bigM,t,n))
